In [1]:
from pykafka import KafkaClient
from pykafka.common import OffsetType
import time


In [2]:
client = KafkaClient("127.0.0.1:9092")

In [6]:
def get_messages(topicname):
    consumer = client.topics[topicname].get_simple_consumer(auto_offset_reset=OffsetType.LATEST, reset_offset_on_start=True)
    offsets = [(p, op.last_offset_consumed)
           for p, op in consumer._partitions.items()]
    offsets = [(p, (o if o > -1 else -2)) for p, o in offsets]
    consumer.reset_offsets(offsets)
    def events():
        for message in consumer:
            msg = message.value.decode()
            if msg!='stop':
                yield f"{time.time()}|topic:{topicname}|message:{msg}"
            else:
                print('stopping')
                break
    return events()

In [7]:
for x in get_messages("test"):
    print(x)

1592439051.5402143|topic:test|message:first message
1592439057.1643996|topic:test|message:second
1592439059.9552042|topic:test|message:another one
1592439069.146359|topic:test|message:stop when message is 'stop'
stopping
